In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.decomposition import PCA

import os

In [2]:
matchStats_file_path = os.getcwd()[:-4] + '\\csvData\\stats\\matchStats.csv'
matchStats_data = pd.read_csv(matchStats_file_path)
matchStats_data

,match_id,team_id,fouls,injuries,corners,offsides,shots_total,shots_on_target,shots_off_target,shots_blocked,...,substitutions,goal_kick,goal_attempts,free_kick,throw_in,ball_safe,goals,penalties,attacks,dangerous_attacks
0,25230,1035,10.0,1.0,7,2,11,3.0,3.0,5.0,...,3,6,6,10,25,0,2,0/1,0,0
1,25230,1052,9.0,0.0,5,1,9,4.0,2.0,3.0,...,3,7,6,12,20,0,3,0/0,0,0
2,25231,1047,20.0,0.0,5,1,8,3.0,2.0,3.0,...,2,16,5,7,24,0,1,0/0,0,0
3,25231,1040,7.0,1.0,7,0,18,3.0,8.0,7.0,...,1,4,11,21,30,0,1,0/0,0,0
4,25232,1038,8.0,2.0,7,2,22,11.0,8.0,3.0,...,3,15,19,8,17,0,4,0/0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,30198,2214,15.0,2.0,3,1,6,4.0,2.0,0.0,...,3,12,6,14,18,0,1,0/0,0,0
12220,30199,2222,19.0,3.0,6,1,22,7.0,12.0,3.0,...,3,8,19,24,16,0,1,0/0,0,0
12221,30199,2213,24.0,0.0,4,0,14,6.0,7.0,1.0,...,2,13,13,20,15,0,2,0/0,0,0
12222,30200,870,15.0,4.0,7,0,19,6.0,8.0,5.0,...,3,3,14,16,18,0,1,1/1,0,0


In [3]:
df = pd.read_csv(matchStats_file_path)
# matchStats_data = matchStats_data.iloc[1:]
# df = df.drop(['match_id', 'team_id'], axis=1)
# df = df.dropna()
df.isnull().sum()
# df

match_id                0
team_id                 0
fouls                2658
injuries             4508
corners                 0
offsides                0
shots_total             0
shots_on_target      2628
shots_off_target     2666
shots_blocked        3172
possessionpercent       0
yellowcards             0
yellowredcards          0
redcards                0
substitutions           0
goal_kick               0
goal_attempts           0
free_kick               0
throw_in                0
ball_safe               0
goals                   0
penalties               0
attacks                 0
dangerous_attacks       0
dtype: int64

In [4]:
df.isnull().sum()

match_id                0
team_id                 0
fouls                2658
injuries             4508
corners                 0
offsides                0
shots_total             0
shots_on_target      2628
shots_off_target     2666
shots_blocked        3172
possessionpercent       0
yellowcards             0
yellowredcards          0
redcards                0
substitutions           0
goal_kick               0
goal_attempts           0
free_kick               0
throw_in                0
ball_safe               0
goals                   0
penalties               0
attacks                 0
dangerous_attacks       0
dtype: int64

In [87]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

# the independent variables set
# X = df[['injuries', 'corners', 'offsides', 
#           'shots_on_target', 'shots_blocked',
#           'yellowcards', 'yellowredcards', 'redcards', 'goal_kick'
#           ]]

# X = df[['fouls', 'injuries', 'shots_on_target', 
#           'shots_off_target', 'shots_blocked',
#           'offsides'
#           ]]


# free_kick
# shots_off_target
# goal_attempts
# shots_total
# fouls
# possessionpercent\
# attacks
# 'throw_in', 'ball_safe', 'dangerous_attacks'
# substitutions

X = df[['match_id', 'team_id', 'fouls', 'injuries', 'corners', 'offsides',
          'shots_on_target', 'shots_off_target', 'shots_blocked',
          'possessionpercent', 'yellowcards', 'yellowredcards', 'redcards',
          'substitutions', 'goal_kick', 'goal_attempts',
          'free_kick', 'throw_in', 'ball_safe', 'attacks', 'dangerous_attacks']] 

# X = df[['injuries', 'corners', 'offsides',
#         'shots_on_target', 'shots_off_target', 'shots_blocked',
#         'yellowcards', 'yellowredcards', 'redcards',
#         'goal_kick',
#         'ball_safe']] 

# 'match_id', 'team_id'
# penalties
# shots_total
# goal_attempts
# possessionpercent
# fouls
# attacks
# free_kick
# throw_in
# dangerous_attacks
# substitutions

X = X.dropna()
X


,match_id,team_id,fouls,injuries,corners,offsides,shots_on_target,shots_off_target,shots_blocked,possessionpercent,...,yellowredcards,redcards,substitutions,goal_kick,goal_attempts,free_kick,throw_in,ball_safe,attacks,dangerous_attacks
0,25230,1035,10.0,1.0,7,2,3.0,3.0,5.0,52,...,0,0,3,6,6,10,25,0,0,0
1,25230,1052,9.0,0.0,5,1,4.0,2.0,3.0,48,...,0,0,3,7,6,12,20,0,0,0
2,25231,1047,20.0,0.0,5,1,3.0,2.0,3.0,45,...,0,0,2,16,5,7,24,0,0,0
3,25231,1040,7.0,1.0,7,0,3.0,8.0,7.0,55,...,0,0,1,4,11,21,30,0,0,0
4,25232,1038,8.0,2.0,7,2,11.0,8.0,3.0,51,...,0,0,3,15,19,8,17,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,30198,2214,15.0,2.0,3,1,4.0,2.0,0.0,48,...,0,0,3,12,6,14,18,0,0,0
12220,30199,2222,19.0,3.0,6,1,7.0,12.0,3.0,49,...,0,0,3,8,19,24,16,0,0,0
12221,30199,2213,24.0,0.0,4,0,6.0,7.0,1.0,51,...,1,0,2,13,13,20,15,0,0,0
12222,30200,870,15.0,4.0,7,0,6.0,8.0,5.0,55,...,0,0,3,3,14,16,18,0,0,0


In [88]:
# VIF dataframe mns
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 

# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print(vif_data)

              feature          VIF
0            match_id     1.406078
1             team_id     2.365259
2               fouls    12.090020
3            injuries     2.300136
4             corners     5.905402
5            offsides     2.259002
6     shots_on_target  1290.929278
7    shots_off_target  1396.692845
8       shots_blocked     3.214315
9   possessionpercent    23.885441
10        yellowcards     3.780990
11     yellowredcards     1.094500
12           redcards     1.073078
13      substitutions     8.286646
14          goal_kick     5.961550
15      goal_attempts  4834.487053
16          free_kick    12.101168
17           throw_in    11.461577
18          ball_safe     9.694150
19            attacks    11.941120
20  dangerous_attacks     9.166162
